In [7]:
from pydantic import BaseModel, Field
from enum import Enum
import requests
from bs4 import BeautifulSoup
import random

from home_lab.defaults import make_chain, make_agent
from langchain.tools import tool
from langfuse.callback import CallbackHandler
from typing import List

In [4]:
class InstrumentType(Enum):
    WOODWIND = "WOODWIND"
    BRASS = "BRASS"
    STRING = "STRING"
    KEYBOARD = "KEYBOARD"

class InstrumentInfo(BaseModel):
    instrument_type: InstrumentType = Field(..., description="what type of instrument it is")
    common_genres: List[str] = Field(..., description="some common genres that use this instrument")

describe_instrument_prompt = """
I am going to give you an instrument, return the type and common genres.

{format_instructions}

{input}
"""

describe_instrument_chain = make_chain(describe_instrument_prompt, InstrumentInfo)
callback_handler = CallbackHandler()
res = describe_instrument_chain.invoke({"input": "guitar"}, config={'callbacks': [callback_handler]})
print(res)

instrument_type=<InstrumentType.STRING: 'STRING'> common_genres=['Rock', 'Pop', 'Blues', 'Country']


In [9]:
@tool
def navigate(url):
    """
    Navigates to a given url and returns all hyperlinks and text.
    Use this to jump from article to article

    Args:
        url (str): The URL to scrape.

    Returns:
        list of tuples: Each tuple contains (link_text, href)
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = []
    for a_tag in soup.find_all('a', href=True):
        text = a_tag.get_text(strip=True)
        href = a_tag['href']
        if href.startswith('/'):
            href = 'https://wikipedia.org' + href
        links.append((text, href))

    return random.sample(links, min(50, len(links)))

class FinalArticle(BaseModel):
    final_article_title: str = Field(..., description="The title of the final article.")
    final_article_url: str = Field(..., description="The url of the final article.")

wiki_prompt = """
I am going to supply you with a random wikipedia article URL.
Your job is to get as close to the target topic by clicking similar links.
Once you hit the max iterations or are satisfied with how close you are to the topic, return the requested info about the final article.

{format_instructions}

starting point: {input}

target topic: {target}
"""

wiki_agent = make_agent([navigate], wiki_prompt, FinalArticle)
callback_handler = CallbackHandler()

seed_url = "https://en.wikipedia.org/wiki/Windows_XP"
callback_handler = CallbackHandler()
res = wiki_agent.invoke(
    {"input": seed_url, "target": "history"}, config={"callbacks": [callback_handler]}
)
print(res)

final_article_title='History' final_article_url='https://en.wikipedia.org/wiki/History'
